In [1]:
import pandas as pd
import os
from easyocr import Reader

reader = Reader(lang_list=['ru', 'en'], gpu=True)
platform_dict = {
    "tg": 1,
    "vk": 2,
    "zn": 3
}
yt = 4
DATASET = "train_dataset"

ALLOWED_FILES = ('.png', ".jpg", ".jpeg")

In [4]:
import torch

In [5]:
torch.cuda.is_available()

True

In [2]:
tg_df = pd.read_excel(f"{DATASET}/tg/tg.xlsx")
vk_df = pd.read_excel(f"{DATASET}/vk/vk.xlsx")
zn_df = pd.read_excel(f"{DATASET}/zn/zn.xlsx")

In [3]:
tg_df.rename({'VR': 'y'}, inplace=True, axis=1)
vk_df.rename({"Количество подписчиков": 'y'}, inplace=True, axis=1)
zn_df.rename({"Количество дочитываний": 'y'}, inplace=True, axis=1)

In [4]:
vk_df

,y,image
0,898,85b23803-d48f-46c8-adf5-b1944865041e.jpg
1,1523,d4f1e1da-99d1-410e-8e2f-214d006a0823.jpg
2,213,0918154b-d999-4978-9216-29e73f27ffed.png
3,248,ce59a28e-1358-402a-9784-fe7d074fa5b1.jpg
4,967,014db9f7-b32d-4eae-a9cd-216f57b3a0f7.docx
...,...,...
710,840,48a4dd38-121d-4741-ad06-ac8887be0ee7.jpg
711,189,700f2946-e271-4c8b-b449-96d42b0977df.jpg
712,12,dd58ff06-7499-4166-8bc8-f5c18f372e1c.png
713,34,f99bdad1-cbd4-4399-8441-8a2dd742c4cb.jpg


In [9]:
import PIL

PIL.__version__

'9.5.0'

In [5]:
df = pd.DataFrame(columns=["text", "platform", "y"])
df

,text,platform,y


In [6]:
def get_images(df: pd.DataFrame) -> list:
    return df['image'].tolist()

In [7]:
from typing import Union


def get_y_value(df: pd.DataFrame, filename) -> Union[int, float]:
    return df[df['image'] == filename]['y'].values[0]

In [8]:
def get_dataframe(platform) -> pd.DataFrame:
    if platform == "vk":
        return vk_df
    elif platform == "zn":
        return zn_df
    else:
        return tg_df

In [9]:
print(vk_df[vk_df['image'] == "0a5dbcdc-2e44-4579-a576-c93d5ee55485.png"]['y'].values[0])

136


In [10]:
from tqdm import tqdm

for platform in platform_dict.keys():
    path = f"{DATASET}/{platform}/images"
    dataframe = get_dataframe(platform)
    files: list[str] = get_images(dataframe)
    for file in tqdm(files):
        if file.lower().endswith(ALLOWED_FILES):
            try:
                text = " ".join(reader.readtext(f"{path}/{file}", detail=0, paragraph=True))
                y = get_y_value(dataframe, file)
                data = {"text": text, "platform": platform_dict[platform], "y": y}
                df = df._append(data, ignore_index=True)
            except:
                continue
df

100%|██████████| 64/64 [01:10<00:00,  1.11s/it]


,text,platform,y
0,pro livedune com LiveDune psy4mylife вовлеченн...,1,108
1,Статистика Telegram-Ka Причеры -криншстов (пс ...,1,75
2,TGStat Быстрый поиск. Мария Петрова Гтари тотр...,1,60
3,TGStat Быстрый поиск Roman Kolymazhnov EE Ката...,1,98
4,09:16 LTE LIVEDUNE Сводная Посты Вовлеченность...,1,86.61
...,...,...,...
1032,0 Ы77 dzen ru profile /editor/id/5ff4de8aaf1 4...,3,13914
1033,dzenru / profile/editor/id/642de8Od345caclfgGe...,3,81
1034,Статистика Системз кснк Дзен Янде Почта MailrL...,3,1080
1035,Дзен-студия Мама; сноEа рисю: Главнсе Статисти...,3,2875


In [11]:
df

,text,platform,y
0,pro livedune com LiveDune psy4mylife вовлеченн...,1,108
1,Статистика Telegram-Ka Причеры -криншстов (пс ...,1,75
2,TGStat Быстрый поиск. Мария Петрова Гтари тотр...,1,60
3,TGStat Быстрый поиск Roman Kolymazhnov EE Ката...,1,98
4,09:16 LTE LIVEDUNE Сводная Посты Вовлеченность...,1,86.61
...,...,...,...
1032,0 Ы77 dzen ru profile /editor/id/5ff4de8aaf1 4...,3,13914
1033,dzenru / profile/editor/id/642de8Od345caclfgGe...,3,81
1034,Статистика Системз кснк Дзен Янде Почта MailrL...,3,1080
1035,Дзен-студия Мама; сноEа рисю: Главнсе Статисти...,3,2875


In [72]:
import re

anchor_words = ["вовлеченность подписчиков", "vr", "вовлеченность подписчиков", "средний охват публикации",]
text:list = df.iloc[6]['text'].lower().split()
new_text = " ".join(text)
for word in anchor_words:
    if word in text:
        index = text.index(word)
        new_text = " ".join(text[index: ])
        
pattern = r'([\d.]+(?:%)?)'
print(pattern)

re.findall(pattern, new_text)


([\d.]+(?:%)?)


['31',
 '21.29',
 '1',
 '1',
 '1',
 '9',
 '7',
 '0',
 '2',
 '0...23.42.13',
 '.',
 '113',
 '0.4',
 '19',
 '53',
 '5',
 '.',
 '13',
 '.',
 '.',
 '121',
 '1',
 '3',
 '0...10.53.24',
 '107.1%',
 '0%',
 '24',
 '1',
 '2',
 '1',
 '529',
 '12.03.2022',
 '09.10.2022',
 '12',
 '84',
 '108%',
 '18.07%',
 '9',
 '2',
 '0..',
 '14.17.06',
 '31']

In [56]:
"|".join(word for word in anchor_words)

'вовлеченность подписчиков|средний охват публикации|vr'

In [71]:
df.iloc[6]

text        Chrome Файл Изменить Посмотреть История Заклад...
platform                                                    1
y                                                         107
Name: 6, dtype: object

In [25]:
df.to_csv("image_dataset.csv", index=False)